In [299]:
import json
from datetime import datetime
import pandas as pd

In [300]:
f = open('all.txt','r')

In [301]:
data = json.load(f)

In [302]:
data.keys

<function dict.keys>

In [303]:
data.keys()

dict_keys(['list', 'items'])

In [304]:
flights = data['items']

In [305]:
swizFlights = [flight for flight in flights if flight['takeoff']['countryIso']=='CH' ]
#swizFlights[0]

In [306]:
date =  swizFlights[0]['pointStart']['time'][:10]
date

'2017-09-29'

In [307]:
name = swizFlights[4]['takeoff']['name']
name

'Rotenflue'

In [308]:
distance = swizFlights[0]['league']['route']['points']
distance

4.18

In [309]:
def getFlightDate(flight):
    date =  flight['pointStart']['time'][:10]
    return date


def strToDate(str):
    date = datetime.strptime(str, '%Y-%m-%d')
    return date


def getCity(flight):
    return flight['takeoff']['name']


def getDistance(flight):
    return flight['league']['route']['points']


In [310]:
#collecting date wise flight data in python dictionary
dateWiseFlights = {}
for flight in swizFlights:
    date = getFlightDate(flight)
    dateDict = dateWiseFlights.get(date,{}) #date as key 
    city = getCity(flight)
    distance = float(getDistance(flight)) 
    cityFlights = dateDict.get(city,{}) #city as key in every date dictionary
    maxFlight = cityFlights.get('max',0) 
    cityFlights['max'] = max(maxFlight,distance)
    totalDistance = cityFlights.get('totalDistance',0)
    cityFlights['totalDistance'] = totalDistance + distance
    cityFlights['totalFlights'] = cityFlights.get('totalFlights',0)+1
    dateDict[city] = cityFlights
    dateWiseFlights[date]  = dateDict

In [311]:
flightsList = sorted(dateWiseFlights.items(), key = lambda x : x[0])

In [312]:
sortedFlights = []
for dateFlight in flightsList:
    dateFlight = list(dateFlight)
    dateFlight[1] = sorted(dateFlight[1].items(), key = lambda x : x[1]['totalDistance'],reverse=True)
    dateFlight[1] = dateFlight[1][0]
    sortedFlights.append(dateFlight)
sortedFlights = [flight for flight in sortedFlights if flight[1][1]['totalFlights']>4 and "2017" in flight[0] ]

In [313]:
import numpy as np

In [314]:
arr = np.empty(shape=(len(sortedFlights),5,))

In [315]:
data = []
for flight in sortedFlights:
    d = []
    d.append(flight[0])
    d.append(flight[1][0])
    d.append(flight[1][1]['max'])
    d.append(flight[1][1]['totalDistance'])
    d.append(flight[1][1]['totalFlights'])
    data.append(d)

In [316]:
data[0]

['2017-01-01', 'Brunni', 22.99, 204.55000000000004, 15]

In [317]:
#saving list of datewise bestspots as pickle
import pickle
with open('flight_data.p', 'wb') as fp:
    pickle.dump(data, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [318]:
all_cities = set() #set of all cities present in the data
for date in list(dateWiseFlights):
    dateData = dateWiseFlights[date]
    cities = list(dateData)
    for city in cities:
        all_cities.add(city)

In [319]:
#removing invalid cities
for city in list(all_cities):
    if(len(city)<4):
        all_cities.remove(city)

In [320]:
#collecting city wise flight data from dateWiseFlight dictionary for every date sorted by date
cityWiseFlights = {} #python dictionary where city is key and for each city value is pandas dataframe containing data for this city for every date
col_lst = ['Date', 'maxDistance', 'avgDistance', 'totalFlights']
dates = list(dateWiseFlights)
dates.sort() #sorting dates
#removing dates 2016 dates as no weather data available
dates = [date for date in dates if "2017" in date]

#collecting all cities data
for city in all_cities:
    city_data_lst = []
    
    for date in dates:
        date_data = dateWiseFlights[date]
        if(city in list(date_data)):  #if data available for this city for this date then save it into the list
            lst = []
            lst.append(date)
            city_data = date_data[city]
            maxDistance = city_data['max']
            totalDistance = city_data['totalDistance']
            totalFlights = city_data['totalFlights']
            if totalFlights>3: #only significant when number of flights greater than 10
                avgDistance = totalDistance/totalFlights
                lst.append(maxDistance)
                lst.append(avgDistance)
                lst.append(totalFlights)
                city_data_lst.append(lst)
    if(len(city_data_lst)>10): #only considering this city when number of days for this city are greater than 10
        city_df = pd.DataFrame(city_data_lst,columns=col_lst)
        cityWiseFlights[city] = city_df
            

In [321]:
city_w = cityWiseFlights['Weissenstein']

In [322]:
#saving citywise data 
import pickle
with open('flight_data_citywise.p', 'wb') as fp:
    pickle.dump(cityWiseFlights, fp, protocol=pickle.HIGHEST_PROTOCOL)
#removing cities with very less data from all cities
all_cities = [city for city in all_cities if city in list(cityWiseFlights)]

In [323]:
#cities sorted by total number of flights in year
sorted_cities = sorted(list(all_cities), key=lambda x: cityWiseFlights[x]['totalFlights'].sum(),reverse=True)
# for city in sorted_cities:
#     print(city, cityWiseFlights[city]['totalFlights'].sum())

In [324]:
#cities sorted by total distance in year
sorted_cities = sorted(list(all_cities), key=lambda x: pd.DataFrame(cityWiseFlights[x]['avgDistance'].values*cityWiseFlights[x]['totalFlights'].values)[0].sum(),reverse=True)
# for city in sorted_cities:
#     print(city, pd.DataFrame(cityWiseFlights[city]['avgDistance'].values*cityWiseFlights[city]['totalFlights'].values)[0].sum())


In [325]:
#cities sorted by maximum of max distance in year
sorted_cities = sorted(list(all_cities), key=lambda x: cityWiseFlights[x]['maxDistance'].max(),reverse=True)
# for city in sorted_cities:
#     print(city, cityWiseFlights[city]['maxDistance'].max())
    

In [326]:
#cities sorted by maximum of average distance in year
sorted_cities = sorted(list(all_cities), key=lambda x: cityWiseFlights[x]['avgDistance'].max(),reverse=True)
# for city in sorted_cities:
#     print(city, cityWiseFlights[city]['avgDistance'].max())

In [327]:
all_cities = sorted(list(all_cities), key=lambda x: cityWiseFlights[x]['totalFlights'].sum(),reverse=True)
#saving all cities
with open('all_cities.p','wb') as fp:
    pickle.dump(all_cities,fp,protocol= pickle.HIGHEST_PROTOCOL)
    

In [328]:

#making dateWise dataframe for each city together if no flight for a city on a date then set all values to zero
param_types = [" maxDistance", " totalDistance", " totalFlights"]
col_list = ["Date"]
for city in all_cities:
    for param in param_types:
        col_list.append(city+param)

        
all_data = []
for date in dates:
    data = []
    data.append(date)
    for city in all_cities:
        if(date in list(cityWiseFlights[city]["Date"])):
            city_data = dateWiseFlights[date][city]
            data.append(city_data["max"])
            totalDistance= city_data["totalDistance"]
            totalFlights = city_data["totalFlights"]
            data.append(totalDistance)
            data.append(totalFlights)
        else:
            data.append(0)
            data.append(0)
            data.append(0)
    all_data.append(data)
date_wise_df_all = pd.DataFrame(all_data,columns = col_list)
date_wise_df_all.head()
    

,Date,Weissenstein maxDistance,Weissenstein totalDistance,Weissenstein totalFlights,Brunni maxDistance,Brunni totalDistance,Brunni totalFlights,Fiesch maxDistance,Fiesch totalDistance,Fiesch totalFlights,...,Grandvillard totalFlights,Kronberg maxDistance,Kronberg totalDistance,Kronberg totalFlights,Pizol maxDistance,Pizol totalDistance,Pizol totalFlights,Gana maxDistance,Gana totalDistance,Gana totalFlights
0,2017-01-01,0.0,0.0,0,22.99,204.55,15,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
1,2017-01-02,0.0,0.0,0,0.00,0.00,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
2,2017-01-03,0.0,0.0,0,0.00,0.00,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
3,2017-01-06,0.0,0.0,0,0.00,0.00,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
4,2017-01-07,0.0,0.0,0,0.00,0.00,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0


In [329]:
date_wise_df_all.shape


(313, 112)

In [330]:
#saving citywise data 
import pickle
with open('flight_data_datewise.p', 'wb') as fp:
    pickle.dump(date_wise_df_all, fp, protocol=pickle.HIGHEST_PROTOCOL)